In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
PATH = "/content/drive/MyDrive/[EXTERNAL] breakthrough_tech_ai_f24/data"

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os

In [4]:
md = md = pd.read_csv(PATH + "/metadata.csv")

In [5]:
md.head()

,bldg_id,in.building_america_climate_zone,in.iso_rto_region,in.state,in.cluster_id,in.cluster_name,in.heating_fuel,in.interior_lighting_generation,in.floor_area_category,in.service_water_heating_fuel,in.comstock_building_type,in.comstock_building_type_group,in.vintage,in.sqft,in.nhgis_county_gisjoin
0,74,Hot-Humid,NaN,AL,53.0,Central Alabama,NaturalGas,gen2_t8_halogen,10001_25000,Electricity,Outpatient,Healthcare,1980 to 1989,17500.0,G0101010
1,115,Hot-Humid,NaN,AL,53.0,Central Alabama,Electricity,gen4_led,10001_25000,Electricity,Outpatient,Healthcare,1960 to 1969,17500.0,G0101010
2,126,Mixed-Humid,NaN,AL,53.0,Central Alabama,Electricity,gen2_t8_halogen,10001_25000,Electricity,Outpatient,Healthcare,1980 to 1989,17500.0,G0100150
3,293,Hot-Humid,NaN,AL,53.0,Central Alabama,NaturalGas,gen2_t8_halogen,10001_25000,NaturalGas,Outpatient,Healthcare,1990 to 1999,17500.0,G0101010
4,313,Mixed-Humid,NaN,AL,53.0,Central Alabama,NaturalGas,gen2_t8_halogen,500001_1mil,NaturalGas,Hospital,Healthcare,1980 to 1989,750000.0,G0100730


In [6]:
# Define the function to check for building files
def get_valid_building_id(building_ids, base_path):
    for building_id in building_ids:
        load_path = os.path.join(base_path, "building_data", str(building_id), "load.csv")
        weather_path = os.path.join(base_path, "building_data", str(building_id), "weather.csv")

        # Check if both load and weather files exist
        if os.path.exists(load_path) and os.path.exists(weather_path):
            return building_id  # Return the first valid building ID

    # Return None if no valid building ID is found
    return None

In [85]:
building_id1a = get_valid_building_id(['5'], PATH)
print("Building_ID: ", building_id1a)

df_load1a = pd.read_csv(PATH + "/building_data/" + str(building_id1a) +"/load.csv")

Building_ID:  5


In [86]:
df_load1a.dtypes

,0
timestamp,object
out.electricity.total.energy_consumption,float64


In [87]:
df_load1a.head()

,timestamp,out.electricity.total.energy_consumption
0,2018-01-01 00:15:00,4.056637
1,2018-01-01 00:30:00,4.056637
2,2018-01-01 00:45:00,4.056637
3,2018-01-01 01:00:00,4.056637
4,2018-01-01 01:15:00,5.186527


In [88]:
def process_time_features(load_series: pd.DataFrame):
    """
    Prepare target matrix Y from a load timeseries DataFrame, ensuring the correct shape with 96 columns
    (for the next 24 hours of 15-minute intervals) and aligned time features.

    Parameters:
        load_series (pd.DataFrame): DataFrame with columns ['timestamp', 'out.electricity.total.energy_consumption'].
                                    Assumed to be sampled at 15-minute intervals.

    Returns:
        pd.DataFrame: Combined DataFrame with the 15-minute interval timestamps, time features,
                      and the target matrix (96 time steps ahead).
    """
    result_df = pd.DataFrame()

    # Convert 'timestamp' to datetime format
    load_series['timestamp'] = pd.to_datetime(load_series['timestamp'])
    result_df['timestamp'] = load_series['timestamp']

    # Extract time-based features from the timestamp
    result_df['hour'] = load_series['timestamp'].dt.hour
    result_df['day_of_week'] = load_series['timestamp'].dt.dayofweek
    result_df['month'] = load_series['timestamp'].dt.month
    result_df['year'] = load_series['timestamp'].dt.year

    # # Cyclic encoding for hour and day_of_week (to account for the cyclical nature)
    # load_series['hour_sin'] = np.sin(2 * np.pi * load_series['hour'] / 24)
    # load_series['hour_cos'] = np.cos(2 * np.pi * load_series['hour'] / 24)
    # load_series['day_of_week_sin'] = np.sin(2 * np.pi * load_series['day_of_week'] / 7)
    # load_series['day_of_week_cos'] = np.cos(2 * np.pi * load_series['day_of_week'] / 7)

    result_df['TotalEnergyConsumption'] = load_series['out.electricity.total.energy_consumption']

    return result_df

In [89]:
target_y = process_time_features(df_load1a)

In [90]:
target_y.shape

(35040, 6)

In [94]:
target_y[:95]

,timestamp,hour,day_of_week,month,year,TotalEnergyConsumption
0,2018-01-01 00:15:00,0,0,1,2018,4.056637
1,2018-01-01 00:30:00,0,0,1,2018,4.056637
2,2018-01-01 00:45:00,0,0,1,2018,4.056637
3,2018-01-01 01:00:00,1,0,1,2018,4.056637
4,2018-01-01 01:15:00,1,0,1,2018,5.186527
...,...,...,...,...,...,...
90,2018-01-01 22:45:00,22,0,1,2018,4.142348
91,2018-01-01 23:00:00,23,0,1,2018,5.197480
92,2018-01-01 23:15:00,23,0,1,2018,5.190304
93,2018-01-01 23:30:00,23,0,1,2018,4.143571


In [57]:
target_y.dtypes

,0
timestamp,datetime64[ns]
out.electricity.total.energy_consumption,float64
hour,int32
day_of_week,int32
minute,int32
month,int32
year,int32
